# newzy_project

In [1]:
import pandas as pd
import nltk
import warnings
warnings.filterwarnings('ignore')

In [2]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('tagsets')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/treyhamilton/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/treyhamilton/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/treyhamilton/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     /Users/treyhamilton/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


True

 F0: Source Format. The initial source format of a text, which varies by
collection, e.g. XML (e.g. TEI and RSS), HTML, plain text (e.g.
Gutenberg), JSON, and CSV.

In [3]:
file = 'newzy.csv'

In [4]:
newzy = pd.read_csv("newzy.csv", sep="|", header=0).set_index('doc_id')

In [5]:
newzy.head()

,doc_source,doc_title,doc_content,doc_date,doc_url
doc_id,,,,,
39,Google News,"50 Years on, Finding Profit in 'Truth' on JFK ...","Washington Post50 Years on, Finding Profit in ...",11/03/2013,http://news.google.com/news/url?sa=t&fd=R&usg=...
38,Google News,"Nearly 10 years later, families and officials ...","Washington PostNearly 10 years later, families...",11/03/2013,http://news.google.com/news/url?sa=t&fd=R&usg=...
37,Google News,"Cuccinelli, running hard, has been hobbled in ...","Washington PostCuccinelli, running hard, has b...",11/03/2013,http://news.google.com/news/url?sa=t&fd=R&usg=...
36,Google News,Chuck Schumer endorses Hillary Clinton for Pre...,New York Daily NewsChuck Schumer endorses Hill...,11/03/2013,http://news.google.com/news/url?sa=t&fd=R&usg=...
35,Google News,Sec. Kerry issues apparent warning to Iran - C...,Ahram OnlineSec. Kerry issues apparent warning...,11/03/2013,http://news.google.com/news/url?sa=t&fd=R&usg=...


In [6]:
newzy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1026347 entries, 39 to 1026382
Data columns (total 5 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   doc_source   1025990 non-null  object
 1   doc_title    1026344 non-null  object
 2   doc_content  986462 non-null   object
 3   doc_date     1025990 non-null  object
 4   doc_url      1025989 non-null  object
dtypes: object(5)
memory usage: 47.0+ MB


In [7]:
newzy = newzy.dropna()

In [8]:
newzy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 986461 entries, 39 to 1026382
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   doc_source   986461 non-null  object
 1   doc_title    986461 non-null  object
 2   doc_content  986461 non-null  object
 3   doc_date     986461 non-null  object
 4   doc_url      986461 non-null  object
dtypes: object(5)
memory usage: 45.2+ MB


In [9]:
newzy['doc_date'] = pd.to_datetime(newzy.doc_date)

In [10]:
newzy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 986461 entries, 39 to 1026382
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   doc_source   986461 non-null  object        
 1   doc_title    986461 non-null  object        
 2   doc_content  986461 non-null  object        
 3   doc_date     986461 non-null  datetime64[ns]
 4   doc_url      986461 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 45.2+ MB


In [11]:
newzy.groupby('doc_source').doc_date.min().astype(str) + ' to ' + newzy.groupby('doc_source').doc_date.max().astype(str)

doc_source
Breitbart              2013-11-03 to 2020-02-27
CNN                    2013-11-03 to 2020-02-27
Daily Kos              2013-11-03 to 2019-10-31
Drudge Report          2013-11-03 to 2020-02-26
Fox                    2013-11-03 to 2020-02-27
Google News            2013-11-03 to 2020-02-27
Guardian               2013-11-03 to 2020-02-26
NPR                    2013-11-03 to 2020-02-27
New York Times         2013-11-03 to 2020-02-27
Politico Magazine      2013-11-19 to 2020-02-27
PowerLine              2013-11-03 to 2020-02-27
Real Clear Politics    2013-11-03 to 2020-02-27
Reuters                2013-11-03 to 2020-02-27
UPI Latest             2013-11-05 to 2020-02-27
US News                2013-11-05 to 2020-02-27
Name: doc_date, dtype: object

In [12]:
# newzy.groupby('doc_source').count()

In [13]:
#how many articles do each news source have?
newzy.groupby('doc_source').count().max(axis=1)

doc_source
Breitbart               10011
CNN                     36184
Daily Kos               12307
Drudge Report           77063
Fox                     58077
Google News             64078
Guardian                12392
NPR                     24708
New York Times          21145
Politico Magazine        3324
PowerLine                9730
Real Clear Politics     34967
Reuters                 18506
UPI Latest              97804
US News                506165
dtype: int64

F1: Machine Learning Corpus Format (MLCF). Ideally a table of
minimum discursive units indexed by document content hierarchy.

In [14]:
#all doc_content hold no more than 1 paragraph.
OHCO = ['doc_source', 'doc_id', 'doc_title', 'sent_num', 'token_num']

In [15]:
def CREATE_LIB_DOC(df, OHCO=OHCO):    
    df.doc_content = df.doc_content.str.strip()
    
    # FIX CHARACTERS TO IMPROVE TOKENIZATION, Professor states this helps nltk parse text.
    df.doc_content = df.doc_content.str.replace('—', ' — ')
    df.doc_content = df.doc_content.str.replace('-', ' - ')
    
    #all doc_content hold no more than 1 paragraph.
    df['doc_content'] = df['doc_content'].str.replace(r'\n', ' ').str.strip()
    df = df[~df['doc_content'].isna()]
    df = df[~df['doc_content'].str.match(r'^\s*$', na=False)] # Remove empty paragraphs
    
    library = df[['doc_source', 'doc_title', 'doc_date', 'doc_url']]
    
    df = df.reset_index().set_index(OHCO[:3])
    df = df.drop(columns=['doc_date', 'doc_url'])
    docs = df
    return library, docs

In [16]:
LIB, DOC = CREATE_LIB_DOC(newzy)

In [17]:
LIB.head(20)

,doc_source,doc_title,doc_date,doc_url
doc_id,,,,
39,Google News,"50 Years on, Finding Profit in 'Truth' on JFK ...",2013-11-03,http://news.google.com/news/url?sa=t&fd=R&usg=...
38,Google News,"Nearly 10 years later, families and officials ...",2013-11-03,http://news.google.com/news/url?sa=t&fd=R&usg=...
37,Google News,"Cuccinelli, running hard, has been hobbled in ...",2013-11-03,http://news.google.com/news/url?sa=t&fd=R&usg=...
36,Google News,Chuck Schumer endorses Hillary Clinton for Pre...,2013-11-03,http://news.google.com/news/url?sa=t&fd=R&usg=...
35,Google News,Sec. Kerry issues apparent warning to Iran - C...,2013-11-03,http://news.google.com/news/url?sa=t&fd=R&usg=...
34,Google News,"Japanese Civil Servant Runs Marathon for Fun, ...",2013-11-03,http://news.google.com/news/url?sa=t&fd=R&usg=...
33,Google News,"ObamaCare architect defends plan, says preside...",2013-11-03,http://news.google.com/news/url?sa=t&fd=R&usg=...
32,Google News,"Prosecutor: LAX suspect shot TSA officer, left...",2013-11-03,http://news.google.com/news/url?sa=t&fd=R&usg=...
31,Google News,Bangladesh finds UK and US accused guilty of w...,2013-11-03,http://news.google.com/news/url?sa=t&fd=R&usg=...


In [18]:
DOC.head(20)

doc_content
doc_source  doc_id doc_title                                                                                            
Google News 39     50 Years on, Finding Profit in 'Truth' on JFK C...  Washington Post50 Years on, Finding Profit in ...
            38     Nearly 10 years later, families and officials s...  Washington PostNearly 10 years later, families...
            37     Cuccinelli, running hard, has been hobbled in r...  Washington PostCuccinelli, running hard, has b...
            36     Chuck Schumer endorses Hillary Clinton for Pres...  New York Daily NewsChuck Schumer endorses Hill...
            35     Sec. Kerry issues apparent warning to Iran - CB...  Ahram OnlineSec. Kerry issues apparent warning...
            34     Japanese Civil Servant Runs Marathon for Fun, N...  New York Daily NewsJapanese Civil Servant Runs...
            33     ObamaCare architect defends plan, says presiden...  Washington PostObamaCare architect defends pla...
            32     Prosecutor: LAX suspect shot TSA officer, left,...  ABC NewsProsecutor: LAX suspect shot TSA offic...
            31     Bangladesh finds UK and US accused guilty of wa...  BBC NewsBangladesh finds UK and US accused gui...
            30     Two French journalists kidnapped and killed in ...  AFPTwo French journalists kidnapped and killed...
            29     eBay apologises for selling Holocaust memorabil...  Telegraph.co.ukeBay apologises for selling Hol...
            28     Sen. Chuck Schumer endorses Clinton for preside...  New York Daily NewsSen. Chuck Schumer endorses...
            27     US has 'scuttled' Taliban talks with drone stri...  IBNLiveUS has 'scuttled' Taliban talks with dr...
            26     A long, and overdue, end near for Virginia's bi...  Washington PostA long, and overdue, end near f...
            25     HealthCare.gov: How political fear was pitted a...  Voice of AmericaHealthCare.gov: How political ...
            24     Kerry sees signs of Egypt moving back towards d...  The GuardianKerry sees signs of Egypt moving b...
            23     Boston bombings change preparations for NYC mar...  NPRBoston bombings change preparations for NYC...
            22     Slain LAX TSA officer remembered as a family ma...  ABC NewsSlain LAX TSA officer remembered as a ...
            40     College presidents find that smaller jobs can b...  Washington PostCollege presidents find that sm...
            41     Pakistani Taliban Names Interim Chief - Voice o...  Telegraph.co.ukPakistani Taliban Names Interim...

In [19]:
#tokenize function
def tokenize(doc_df, OHCO=OHCO, remove_pos_tuple=False, ws=False):
    
    # Paragraphs to Sentences
    df = doc_df.doc_content\
        .apply(lambda x: pd.Series(nltk.sent_tokenize(x)))\
        .stack()\
        .to_frame()\
        .rename(columns={0:'sent_str'})
    
    # Sentences to Tokens
    # Local function to pick tokenizer
    def word_tokenize(x):
        if ws:
            s = pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x)))
        else:
            s = pd.Series(nltk.pos_tag(nltk.word_tokenize(x)))
        return s
            
    df = df.sent_str\
        .apply(word_tokenize)\
        .stack()\
        .to_frame()\
        .rename(columns={0:'pos_tuple'})
    
    # Grab info from tuple
    df['pos'] = df.pos_tuple.apply(lambda x: x[1])
    df['token_str'] = df.pos_tuple.apply(lambda x: x[0])
    if remove_pos_tuple:
        df = df.drop('pos_tuple', 1)
    
    # Add index
    df.index.names = OHCO
    
    return df

In [ ]:
TOKEN = tokenize(DOC, ws = True)

In [ ]:
TOKEN.head()

In [ ]:
#lowercase token and replace any non-alphanumeric character with ""
TOKEN['term_str'] = TOKEN['token_str'].str.lower().str.replace('[\W_]', '')
TOKEN

In [ ]:
TOKEN = TOKEN.dropna()
TOKEN

In [ ]:
#count the number of distinct terms and the number of times they occur and put them into a new dataframe.
VOCAB = TOKEN.term_str.value_counts().to_frame().rename(columns={'index':'term_str', 'term_str':'n'})\
    .sort_index().reset_index().rename(columns={'index':'term_str'})
VOCAB.index.name = 'term_id'

#Check if term is a int
VOCAB['num'] = VOCAB.term_str.str.match("\d+").astype('int')

In [ ]:
VOCAB

In [ ]:
#Create a dataframe of stop words
sw = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str'])
sw = sw.reset_index().set_index('term_str')
sw.columns = ['dummy']
sw.dummy = 1

In [ ]:
#map stop words with terms to determine which words are stop words.
VOCAB['stop'] = VOCAB.term_str.map(sw.dummy)
VOCAB['stop'] = VOCAB['stop'].fillna(0).astype('int')

In [ ]:
VOCAB[VOCAB.stop == 1].sample(10)

In [ ]:
from nltk.stem.porter import PorterStemmer

#determine stems of all words
stemmer = PorterStemmer()
VOCAB['p_stem'] = VOCAB.term_str.apply(stemmer.stem)

In [ ]:
VOCAB.sample(10)

In [ ]:
TOKEN['term_id'] = TOKEN.term_str.map(VOCAB.reset_index().set_index('term_str').term_id)
TOKEN

In [ ]:
#contains the most frequently associated part-of-speech tag, as found in the TOKEN table, with each term.
VOCAB['pos_max'] = TOKEN.groupby(['term_id', 'pos']).count().iloc[:,0].unstack().idxmax(1)

In [ ]:
VOCAB.sample(10)

In [ ]:
#save as csv
#when these are saved as csv, certain term strings don't get saved properly and turn N/A. Will try to avoid reloading data but this does happens if you do reload. Not sure how to fix.
DOC.to_csv('DOC.csv')
LIB.to_csv('LIB.csv')
VOCAB.to_csv('VOCAB.csv')
TOKEN.to_csv('TOKEN.csv')